# Load Oxford Street data

In [5]:
import certifi
certifi.where()

'C:\\Users\\Raphaela\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\certifi\\cacert.pem'

In [1]:
pip install cv2

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)
ERROR: No matching distribution found for cv2


In [1]:
import cv2
import pandas as pd

ModuleNotFoundError: No module named 'cv2'

In [ ]:

# Replace 'path_to_video_file' with the actual path to your video file
video_path = 'path_to_video_file'

# Open the video file
cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    # Process each frame (e.g., display, save, etc.)
    cv2.imshow('Frame', frame)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
